In [1]:
import pandas as pd

In [2]:
train_df = pd.read_json('data/train.json', orient='index')

In [3]:
train_df

,target,features
user_1,female,"{'orders': [{'site-id': 1, 'orders': [{'create..."
user_2,female,"{'visits': [{'site-id': 3, 'first-seen': 16962..."
user_3,male,"{'orders': [{'site-id': 21, 'orders': [{'creat..."
user_4,male,"{'orders': [{'site-id': 2, 'orders': [{'create..."
user_5,female,"{'orders': [{'site-id': 39, 'orders': [{'creat..."
...,...,...
user_127751,male,"{'orders': [{'site-id': 307, 'orders': [{'crea..."
user_127752,male,"{'orders': [{'site-id': 147, 'orders': [{'crea..."
user_127753,female,"{'orders': [{'site-id': 20, 'orders': [{'creat..."
user_127754,female,"{'orders': [{'site-id': 407, 'orders': [{'crea..."


In [4]:
train_df.loc[:,'orders'] = train_df.loc[:,'features'].apply(lambda x: x['orders'] if 'orders' in x else 0)
train_df = train_df.drop('features', axis=1)
train_df

,target,orders
user_1,female,"[{'site-id': 1, 'orders': [{'created-at': 1634..."
user_2,female,0
user_3,male,"[{'site-id': 21, 'orders': [{'created-at': 168..."
user_4,male,"[{'site-id': 2, 'orders': [{'created-at': 1600..."
user_5,female,"[{'site-id': 39, 'orders': [{'created-at': 164..."
...,...,...
user_127751,male,"[{'site-id': 307, 'orders': [{'created-at': 16..."
user_127752,male,"[{'site-id': 147, 'orders': [{'created-at': 16..."
user_127753,female,"[{'site-id': 20, 'orders': [{'created-at': 169..."
user_127754,female,"[{'site-id': 407, 'orders': [{'created-at': 16..."


In [5]:
def get_orders_df(data_df):
    def add_record(user_id, site_id, created_at, item_id, count,
                   general_category_path, brand_id, target=None):
        orders_data['user-id'].append(user_id)
        orders_data['site-id'].append(site_id)
        orders_data['created-at'].append(created_at)
        orders_data['item-id'].append(item_id)
        orders_data['count'].append(count)
        orders_data['general-category-path'].append(general_category_path)
        orders_data['brand-id'].append(brand_id)
        orders_data['target'].append(target)

    orders_data = {'user-id': [], 'site-id': [], 'created-at': [], 'item-id': [], 'count': [],
                   'general-category-path': [], 'brand-id': [], 'target': []}
    users_list = data_df.index
    for i in range(data_df.shape[0]):
        if data_df['orders'][i] == 0: continue
        user_id = users_list[i]
        if data_df['target'][i] == 'male':
            target = 1
        else:
            target = 0
        for site_orders in data_df['orders'][i]:            
            site_id = site_orders['site-id']
            for order in site_orders['orders']:
                created_at = order.get('created-at', None)
                if order['items']:
                    for item in order['items']:
                        item_id = item.get('id', None)
                        count = item.get('count', None)
                        general_category_path = item.get('general-category-path', None)
                        brand_id = item.get('brand-id', None)
                        add_record(user_id, site_id, created_at, item_id, count,
                                    general_category_path, brand_id, target)                        
                else:
                    item_id = None
                    count = None
                    general_category_path = None
                    brand_id = None
                    add_record(user_id, site_id, created_at, item_id, count,
                                general_category_path, brand_id, target)
    return(orders_data)


orders_df = pd.DataFrame(get_orders_df(train_df))

In [6]:
orders_df.head(20)

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target
0,user_1,1,1634292444,None,NaN,None,NaN,0
1,user_1,2,1639827297,None,NaN,None,NaN,0
2,user_1,2,1639827681,None,NaN,None,NaN,0
3,user_3,21,1684248081,None,NaN,None,NaN,1
4,user_3,22,1684892551,item_1,1.0,None,NaN,1
5,user_4,2,1600677045,item_8,1.0,None,NaN,1
6,user_4,29,1611987662,item_9,1.0,"[90404, 90403, 90402]",3243.0,1
7,user_4,29,1655426430,item_10,1.0,"[90698, 7330336, 90692, 90666]",1210537.0,1
8,user_4,29,1655426430,item_11,1.0,"[8353924, 91497, 91461, 198119]",1773.0,1
9,user_4,30,1681785620,None,NaN,None,NaN,1


In [34]:
orders_df.to_csv('data/orders_train.csv')

In [37]:
pd.read_csv('data/orders_val.csv', index_col=0)

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target
0,user_127756,407,1697125567,item_676374,NaN,NaN,NaN,0
1,user_127757,16,1636574129,item_3780263,2.0,"[90578, 90574, 198118]",1237.0,1
2,user_127757,16,1637512604,item_5403882,2.0,"[90578, 90574, 198118]",1237.0,1
3,user_127757,16,1637948102,item_1627033,1.0,"[91498, 91497, 91461, 198119]",1.0,1
4,user_127757,16,1638631112,item_554781,1.0,"[90629, 90625, 10599873, 198119]",3519.0,1
...,...,...,...,...,...,...,...,...
812878,user_155202,53,1684604189,item_650360,1.0,NaN,NaN,0
812879,user_155202,53,1684604189,item_650361,1.0,NaN,NaN,0
812880,user_155202,54,1420434776,NaN,NaN,NaN,NaN,0
812881,user_155202,300,1688068054,item_650362,1.0,"[818863, 15068776, 7811881, 7877999]",915491.0,0
